In [1]:
import pandas as pd
import numpy as np

import sys
sys.path.insert(0, '../')
from SimRank import SimRank

## SimRank on Directive Graphs

### Prepare data
Data used in this section: BTS Flight dataset(https://www.transtats.bts.gov/DL_SelectFields.asp?Table_ID=236)

In [2]:
"""
Load data
keep only top 10 ORIGIN_AIRPORT_ID with the largest airline counts, and top 10 DEST_AIRPORT_ID with the largest airline counts
Count total number of flights for each arilines
"""
flights = pd.read_csv('data/flights.csv')
origin = flights.groupby('ORIGIN_AIRPORT_ID')['DEST_AIRPORT_ID'].count().nlargest(10).index
dest = flights.groupby('DEST_AIRPORT_ID')['ORIGIN_AIRPORT_ID'].count().nlargest(10).index
sample_data1 = flights[flights.ORIGIN_AIRPORT_ID.isin(origin) & flights.DEST_AIRPORT_ID.isin(dest)][['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']]
sample_data1['flights'] = 1
sample_data1 = sample_data1.groupby(['ORIGIN_AIRPORT_ID', 'DEST_AIRPORT_ID']).flights.count().to_frame().reset_index()
sample_data1

,ORIGIN_AIRPORT_ID,DEST_AIRPORT_ID,flights
0,10397,11057,506
1,10397,11292,466
2,10397,11298,508
3,10397,12266,465
4,10397,12889,387
...,...,...,...
79,14107,11298,397
80,14107,12266,246
81,14107,12889,584
82,14107,12892,561


### Simple SimRank on Directive Graphs
Use the SimRank class

In [3]:
"""
Init instance of SimRank class to run the basic SimRank on the directive graph
Not considering weights(flights) on this data sample ended up in an barely differentiated similarity results
("weight_column" is not required if "weighted" is set to False)
"""
sr = SimRank.SimRank()
s_airports = sr.fit(
    sample_data1, 
    weighted = False, 
    from_node_column = 'ORIGIN_AIRPORT_ID', 
    to_node_column = 'DEST_AIRPORT_ID')

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 22

In [4]:
s_airports

,12953,12889,14107,11292,10397,12892,11298,12266,13930,11057
12953,1.000000,0.298554,0.298554,0.279489,0.279489,0.298554,0.279489,0.279489,0.279489,0.279489
12889,0.298554,1.000000,0.289676,0.282750,0.282750,0.289676,0.282750,0.282750,0.282750,0.282750
14107,0.298554,0.289676,1.000000,0.282750,0.282750,0.289676,0.282750,0.282750,0.282750,0.282750
11292,0.279489,0.282750,0.282750,1.000000,0.284122,0.282750,0.284122,0.284122,0.284122,0.284122
10397,0.279489,0.282750,0.282750,0.284122,1.000000,0.282750,0.284122,0.284122,0.284122,0.284122
12892,0.298554,0.289676,0.289676,0.282750,0.282750,1.000000,0.282750,0.282750,0.282750,0.282750
11298,0.279489,0.282750,0.282750,0.284122,0.284122,0.282750,1.000000,0.284122,0.284122,0.284122
12266,0.279489,0.282750,0.282750,0.284122,0.284122,0.282750,0.284122,1.000000,0.284122,0.284122
13930,0.279489,0.282750,0.282750,0.284122,0.284122,0.282750,0.284122,0.284122,1.000000,0.284122
11057,0.279489,0.282750,0.282750,0.284122,0.284122,0.282750,0.284122,0.284122,0.284122,1.000000


In [5]:
"""
Init instance of SimRank class to run the basic SimRank on the directive graph
Now, use the count of flights as the weight and run the SimRank again
"""
sr = SimRank.SimRank()
s_airports = sr.fit(
    sample_data1, 
    weighted = True, 
    from_node_column = 'ORIGIN_AIRPORT_ID', 
    to_node_column = 'DEST_AIRPORT_ID',
    weight_column = 'flights')

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 1

In [6]:
s_airports

,12953,12889,14107,11292,10397,12892,11298,12266,13930,11057
12953,1.000000e+00,3.465807e-07,3.890822e-07,2.685506e-07,2.476391e-07,3.010631e-07,2.628607e-07,3.465275e-07,2.291428e-07,3.944747e-07
12889,3.465807e-07,1.000000e+00,4.114278e-07,3.407687e-07,3.142337e-07,3.183537e-07,3.335487e-07,4.397150e-07,2.907634e-07,5.005561e-07
14107,3.890822e-07,4.114278e-07,1.000000e+00,3.825574e-07,3.527684e-07,3.573936e-07,3.744519e-07,4.936375e-07,3.264200e-07,5.619396e-07
11292,2.685506e-07,3.407687e-07,3.825574e-07,1.000000e+00,3.339240e-07,2.960144e-07,3.544493e-07,4.672682e-07,3.089831e-07,5.319216e-07
10397,2.476391e-07,3.142337e-07,3.527684e-07,3.339240e-07,1.000000e+00,2.729643e-07,3.268490e-07,4.308829e-07,2.849232e-07,4.905020e-07
12892,3.010631e-07,3.183537e-07,3.573936e-07,2.960144e-07,2.729643e-07,1.000000e+00,2.897426e-07,3.819658e-07,2.525765e-07,4.348164e-07
11298,2.628607e-07,3.335487e-07,3.744519e-07,3.544493e-07,3.268490e-07,2.897426e-07,1.000000e+00,4.573679e-07,3.024365e-07,5.206516e-07
12266,3.465275e-07,4.397150e-07,4.936375e-07,4.672682e-07,4.308829e-07,3.819658e-07,4.573679e-07,1.000000e+00,3.987001e-07,6.863715e-07
13930,2.291428e-07,2.907634e-07,3.264200e-07,3.089831e-07,2.849232e-07,2.525765e-07,3.024365e-07,3.987001e-07,1.000000e+00,4.538661e-07
11057,3.944747e-07,5.005561e-07,5.619396e-07,5.319216e-07,4.905020e-07,4.348164e-07,5.206516e-07,6.863715e-07,4.538661e-07,1.000000e+00


### SimRank++ on Directive Graphs
Use the SimRankPP class

In [7]:
"""
Init instance of SimRankPP class to run the SimRank++ on the directive graph
Weight matrixs and Evidence matrixs will be included during the iteration
Not considering weights(flights) on this data sample ended up in an barely differentiated similarity results
("weight_column" is not required if "weighted" is set to False)
"""
srpp = SimRank.SimRankPP()
s_airports = sr.fit(
    sample_data1, 
    weighted = False, 
    from_node_column = 'ORIGIN_AIRPORT_ID', 
    to_node_column = 'DEST_AIRPORT_ID')

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 22

In [8]:
s_airports

,12953,12889,14107,11292,10397,12892,11298,12266,13930,11057
12953,1.000000,0.298554,0.298554,0.279489,0.279489,0.298554,0.279489,0.279489,0.279489,0.279489
12889,0.298554,1.000000,0.289676,0.282750,0.282750,0.289676,0.282750,0.282750,0.282750,0.282750
14107,0.298554,0.289676,1.000000,0.282750,0.282750,0.289676,0.282750,0.282750,0.282750,0.282750
11292,0.279489,0.282750,0.282750,1.000000,0.284122,0.282750,0.284122,0.284122,0.284122,0.284122
10397,0.279489,0.282750,0.282750,0.284122,1.000000,0.282750,0.284122,0.284122,0.284122,0.284122
12892,0.298554,0.289676,0.289676,0.282750,0.282750,1.000000,0.282750,0.282750,0.282750,0.282750
11298,0.279489,0.282750,0.282750,0.284122,0.284122,0.282750,1.000000,0.284122,0.284122,0.284122
12266,0.279489,0.282750,0.282750,0.284122,0.284122,0.282750,0.284122,1.000000,0.284122,0.284122
13930,0.279489,0.282750,0.282750,0.284122,0.284122,0.282750,0.284122,0.284122,1.000000,0.284122
11057,0.279489,0.282750,0.282750,0.284122,0.284122,0.282750,0.284122,0.284122,0.284122,1.000000


In [9]:
"""
Init instance of SimRankPP class to run the SimRank++ on the directive graph
Weight matrixs and Evidence matrixs will be included during the iteration
Now, use the count of flights as the weight and run the SimRank again
"""
srpp = SimRank.SimRankPP()
s_airports = srpp.fit(
    sample_data1, 
    weighted = True, 
    from_node_column = 'ORIGIN_AIRPORT_ID', 
    to_node_column = 'DEST_AIRPORT_ID',
    weight_column = 'flights')

Initializing Weight matrix...
Finished in 0.0029380321502685547s!
Initializing Evidence matrix...
Finished in 0.001386880874633789s!
Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 1

In [10]:
s_airports

,12953,12889,14107,11292,10397,12892,11298,12266,13930,11057
12953,1.000000e+00,3.411654e-07,3.830028e-07,2.601584e-07,2.399004e-07,2.963590e-07,2.546463e-07,3.356986e-07,2.219821e-07,3.821474e-07
12889,3.411654e-07,1.000000e+00,4.082135e-07,3.381064e-07,3.117788e-07,3.158665e-07,3.309428e-07,4.362797e-07,2.884918e-07,4.966455e-07
14107,3.830028e-07,4.082135e-07,1.000000e+00,3.795686e-07,3.500124e-07,3.546014e-07,3.715265e-07,4.897810e-07,3.238698e-07,5.575494e-07
11292,2.601584e-07,3.381064e-07,3.795686e-07,1.000000e+00,3.326196e-07,2.937018e-07,3.530647e-07,4.654429e-07,3.077761e-07,5.298438e-07
10397,2.399004e-07,3.117788e-07,3.500124e-07,3.326196e-07,1.000000e+00,2.708318e-07,3.255723e-07,4.291998e-07,2.838102e-07,4.885859e-07
12892,2.963590e-07,3.158665e-07,3.546014e-07,2.937018e-07,2.708318e-07,1.000000e+00,2.874790e-07,3.789817e-07,2.506033e-07,4.314194e-07
11298,2.546463e-07,3.309428e-07,3.715265e-07,3.530647e-07,3.255723e-07,2.874790e-07,1.000000e+00,4.555814e-07,3.012551e-07,5.186178e-07
12266,3.356986e-07,4.362797e-07,4.897810e-07,4.654429e-07,4.291998e-07,3.789817e-07,4.555814e-07,1.000000e+00,3.971427e-07,6.836904e-07
13930,2.219821e-07,2.884918e-07,3.238698e-07,3.077761e-07,2.838102e-07,2.506033e-07,3.012551e-07,3.971427e-07,1.000000e+00,4.520932e-07
11057,3.821474e-07,4.966455e-07,5.575494e-07,5.298438e-07,4.885859e-07,4.314194e-07,5.186178e-07,6.836904e-07,4.520932e-07,1.000000e+00


## SimRank on Bipartitle Graphs

### Prepare data
Data used in this section: MovieLens(https://grouplens.org/datasets/movielens/)

In [11]:
"""
Load data
keep only top 10 users with the largest rating counts, and top 10 movies with the largest rating counts
"""
ratings = pd.read_csv('data/ratings.csv')
users = ratings.groupby('userId')['movieId'].count().nlargest(10).index
movies = ratings.groupby('movieId')['userId'].count().nlargest(10).index
sample_data2 = ratings[ratings.userId.isin(users) & ratings.movieId.isin(movies)]
sample_data2

,userId,movieId,rating,timestamp
1225858,8405,110,4.0,1160464030
1225954,8405,260,5.0,1160463829
1225976,8405,296,5.0,1160463818
1225992,8405,318,4.5,1160463940
1226021,8405,356,4.5,1344220422
...,...,...,...,...
19063141,131904,480,4.0,1241145479
19063152,131904,527,4.0,1265933408
19063168,131904,589,4.0,1241145534
19063171,131904,593,4.0,1239074621


### Simple SimRank on Bipartitle Graphs
Use the BipartiteSimRank class

In [12]:
"""
Init instance of BipartiteSimRank class to run the basic SimRank on the bipartitle graph
Not considering weights(rating) on this data sample ended up in an undifferentiated similarity results
That's because every user viewed exactly the same movies and vice versa
("weight_column" is not required if "weighted" is set to False)
"""
bsr = SimRank.BipartiteSimRank()
s_users, s_movies = bsr.fit(
    sample_data2, 
    weighted = False, 
    node_group1_column = 'userId', 
    node_group2_column = 'movieId', 
    weight_column = 'rating')

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 13

In [13]:
s_users

,131904,125794,34576,82418,83090,59477,8405,118205,74142,121535
131904,1.000000,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637
125794,0.285637,1.000000,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637
34576,0.285637,0.285637,1.000000,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637
82418,0.285637,0.285637,0.285637,1.000000,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637
83090,0.285637,0.285637,0.285637,0.285637,1.000000,0.285637,0.285637,0.285637,0.285637,0.285637
59477,0.285637,0.285637,0.285637,0.285637,0.285637,1.000000,0.285637,0.285637,0.285637,0.285637
8405,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,1.000000,0.285637,0.285637,0.285637
118205,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,1.000000,0.285637,0.285637
74142,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,1.000000,0.285637
121535,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,0.285637,1.000000


In [14]:
s_movies

,480,260,356,296,2571,589,110,527,593,318
480,1.000000,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658
260,0.285658,1.000000,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658
356,0.285658,0.285658,1.000000,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658
296,0.285658,0.285658,0.285658,1.000000,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658
2571,0.285658,0.285658,0.285658,0.285658,1.000000,0.285658,0.285658,0.285658,0.285658,0.285658
589,0.285658,0.285658,0.285658,0.285658,0.285658,1.000000,0.285658,0.285658,0.285658,0.285658
110,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,1.000000,0.285658,0.285658,0.285658
527,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,1.000000,0.285658,0.285658
593,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,1.000000,0.285658
318,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,0.285658,1.000000


In [15]:
"""
Init instance of BipartiteSimRank class to run the basic SimRank on the bipartitle graph
Considering weights(rating) this time
And we have differentiated results now
"""
bsr = SimRank.BipartiteSimRank()
s_users, s_movies = bsr.fit(
    sample_data2, 
    weighted = True, 
    node_group1_column = 'userId', 
    node_group2_column = 'movieId', 
    weight_column = 'rating')

Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 3

In [16]:
s_users

,131904,125794,34576,82418,83090,59477,8405,118205,74142,121535
131904,1.000000,0.004384,0.004178,0.004735,0.004129,0.004553,0.004384,0.004495,0.003551,0.004439
125794,0.004384,1.000000,0.004848,0.005494,0.004792,0.005283,0.005087,0.005216,0.004121,0.005151
34576,0.004178,0.004848,1.000000,0.005236,0.004566,0.005035,0.004848,0.004971,0.003927,0.004909
82418,0.004735,0.005494,0.005236,1.000000,0.005175,0.005706,0.005494,0.005634,0.004451,0.005563
83090,0.004129,0.004792,0.004566,0.005175,1.000000,0.004976,0.004792,0.004913,0.003881,0.004852
59477,0.004553,0.005283,0.005035,0.005706,0.004976,1.000000,0.005283,0.005417,0.004279,0.005349
8405,0.004384,0.005087,0.004848,0.005494,0.004792,0.005283,1.000000,0.005216,0.004121,0.005151
118205,0.004495,0.005216,0.004971,0.005634,0.004913,0.005417,0.005216,1.000000,0.004225,0.005281
74142,0.003551,0.004121,0.003927,0.004451,0.003881,0.004279,0.004121,0.004225,1.000000,0.004172
121535,0.004439,0.005151,0.004909,0.005563,0.004852,0.005349,0.005151,0.005281,0.004172,1.000000


In [17]:
s_movies

,480,260,356,296,2571,589,110,527,593,318
480,1.000000,0.005161,0.004713,0.004871,0.005859,0.005631,0.004871,0.005161,0.005224,0.004817
260,0.005161,1.000000,0.004320,0.004466,0.005371,0.005161,0.004466,0.004731,0.004788,0.004416
356,0.004713,0.004320,1.000000,0.004077,0.004904,0.004713,0.004077,0.004320,0.004372,0.004032
296,0.004871,0.004466,0.004077,1.000000,0.005069,0.004871,0.004215,0.004466,0.004519,0.004168
2571,0.005859,0.005371,0.004904,0.005069,1.000000,0.005859,0.005069,0.005371,0.005435,0.005013
589,0.005631,0.005161,0.004713,0.004871,0.005859,1.000000,0.004871,0.005161,0.005224,0.004817
110,0.004871,0.004466,0.004077,0.004215,0.005069,0.004871,1.000000,0.004466,0.004519,0.004168
527,0.005161,0.004731,0.004320,0.004466,0.005371,0.005161,0.004466,1.000000,0.004788,0.004416
593,0.005224,0.004788,0.004372,0.004519,0.005435,0.005224,0.004519,0.004788,1.000000,0.004469
318,0.004817,0.004416,0.004032,0.004168,0.005013,0.004817,0.004168,0.004416,0.004469,1.000000


### SimRank++ on Bipartitle Graphs
The BipartiteSimRankPP class will calculate the Evidence and Weight matrix in order to run the SimRank++ algorithm

In [18]:
"""
Init instance of BipartiteSimRankPP class to run the SimRank++ on the bipartitle graph
Weight matrixs and Evidence matrixs will be included during the iteration
Not considering weights(rating) on this data sample ended up in an undifferentiated similarity results
That's because every user viewed exactly the same movies and vice versa
("weight_column" is not required if "weighted" is set to False)
"""
bsrpp = SimRank.BipartiteSimRankPP()
s_users, s_movies = bsrpp.fit(
    sample_data2, 
    weighted = False, 
    node_group1_column = 'userId', 
    node_group2_column = 'movieId', 
    weight_column = 'rating')

Initializing Weight matrix...
Finished in 0.0013289451599121094s!
Initializing Weight matrix...
Finished in 0.0012862682342529297s!
Initializing Evidence matrix...
Finished in 0.001260995864868164s!
Initializing Evidence matrix...
Finished in 0.0012061595916748047s!
Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 13

In [19]:
s_users

,131904,125794,34576,82418,83090,59477,8405,118205,74142,121535
131904,1.000000,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645
125794,0.284645,1.000000,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645
34576,0.284645,0.284645,1.000000,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645
82418,0.284645,0.284645,0.284645,1.000000,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645
83090,0.284645,0.284645,0.284645,0.284645,1.000000,0.284645,0.284645,0.284645,0.284645,0.284645
59477,0.284645,0.284645,0.284645,0.284645,0.284645,1.000000,0.284645,0.284645,0.284645,0.284645
8405,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,1.000000,0.284645,0.284645,0.284645
118205,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,1.000000,0.284645,0.284645
74142,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,1.000000,0.284645
121535,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,0.284645,1.000000


In [20]:
s_movies

,480,260,356,296,2571,589,110,527,593,318
480,1.000000,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666
260,0.284666,1.000000,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666
356,0.284666,0.284666,1.000000,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666
296,0.284666,0.284666,0.284666,1.000000,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666
2571,0.284666,0.284666,0.284666,0.284666,1.000000,0.284666,0.284666,0.284666,0.284666,0.284666
589,0.284666,0.284666,0.284666,0.284666,0.284666,1.000000,0.284666,0.284666,0.284666,0.284666
110,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,1.000000,0.284666,0.284666,0.284666
527,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,1.000000,0.284666,0.284666
593,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,1.000000,0.284666
318,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,0.284666,1.000000


In [21]:
"""
Init instance of BipartiteSimRankPP class to run the SimRank++ on the bipartitle graph
Weight matrixs and Evidence matrixs will be included during the iteration
Considering weights(rating) this time
And we have differentiated results now
"""
bsrpp = SimRank.BipartiteSimRankPP()
s_users, s_movies = bsrpp.fit(
    sample_data2, 
    weighted = True, 
    node_group1_column = 'userId', 
    node_group2_column = 'movieId', 
    weight_column = 'rating')

Initializing Weight matrix...
Finished in 0.001653909683227539s!
Initializing Weight matrix...
Finished in 0.0015270709991455078s!
Initializing Evidence matrix...
Finished in 0.0012881755828857422s!
Initializing Evidence matrix...
Finished in 0.0031511783599853516s!
Start iterating...
Percent: [##############################] 100% Complete! 
Converged at iteration 3

In [22]:
s_users

,131904,125794,34576,82418,83090,59477,8405,118205,74142,121535
131904,1.000000,0.004379,0.004173,0.004730,0.004125,0.004548,0.004379,0.004490,0.003547,0.004434
125794,0.004379,1.000000,0.004843,0.005489,0.004787,0.005278,0.005082,0.005211,0.004117,0.005146
34576,0.004173,0.004843,1.000000,0.005231,0.004562,0.005029,0.004843,0.004966,0.003923,0.004904
82418,0.004730,0.005489,0.005231,1.000000,0.005170,0.005700,0.005489,0.005628,0.004446,0.005558
83090,0.004125,0.004787,0.004562,0.005170,1.000000,0.004971,0.004787,0.004908,0.003877,0.004847
59477,0.004548,0.005278,0.005029,0.005700,0.004971,1.000000,0.005278,0.005411,0.004275,0.005344
8405,0.004379,0.005082,0.004843,0.005489,0.004787,0.005278,1.000000,0.005211,0.004117,0.005146
118205,0.004490,0.005211,0.004966,0.005628,0.004908,0.005411,0.005211,1.000000,0.004221,0.005276
74142,0.003547,0.004117,0.003923,0.004446,0.003877,0.004275,0.004117,0.004221,1.000000,0.004168
121535,0.004434,0.005146,0.004904,0.005558,0.004847,0.005344,0.005146,0.005276,0.004168,1.000000


In [23]:
s_movies

,480,260,356,296,2571,589,110,527,593,318
480,1.000000,0.005156,0.004708,0.004866,0.005853,0.005625,0.004866,0.005156,0.005218,0.004812
260,0.005156,1.000000,0.004315,0.004461,0.005365,0.005156,0.004461,0.004726,0.004783,0.004411
356,0.004708,0.004315,1.000000,0.004073,0.004899,0.004708,0.004073,0.004315,0.004367,0.004028
296,0.004866,0.004461,0.004073,1.000000,0.005064,0.004866,0.004210,0.004461,0.004515,0.004164
2571,0.005853,0.005365,0.004899,0.005064,1.000000,0.005853,0.005064,0.005365,0.005430,0.005008
589,0.005625,0.005156,0.004708,0.004866,0.005853,1.000000,0.004866,0.005156,0.005218,0.004812
110,0.004866,0.004461,0.004073,0.004210,0.005064,0.004866,1.000000,0.004461,0.004515,0.004164
527,0.005156,0.004726,0.004315,0.004461,0.005365,0.005156,0.004461,1.000000,0.004783,0.004411
593,0.005218,0.004783,0.004367,0.004515,0.005430,0.005218,0.004515,0.004783,1.000000,0.004465
318,0.004812,0.004411,0.004028,0.004164,0.005008,0.004812,0.004164,0.004411,0.004465,1.000000
